<a href="https://colab.research.google.com/github/PingPingE/Learn_ML_DL/blob/main/Practice/Hands_On_ML/ch15-2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import pandas as pd

# 긴 시퀀스 다루기
- 문제점 및 해결 방법(ch15-1에서 왜 tanh가 디폴트인지 알아보면서 살짝 다룸)
  - gradient 소실 또는 폭주 문제 -> 불안정한 학습
  - RNN이 긴 시퀀스를 처리할 때 입력의 첫 부분을 조금씩 잊어버리는 문제 
  - 해결 방법
    - [gradient clipping](https://sanghyu.tistory.com/87)
    - 낮은 learning rate
    - 정규화 
    - 장기 메모리 셀
<br><br>
- RNN에 잘맞는 종류의 정규화: <strong>층 정규화</strong>(ch13-2에서 살짝 다룸)
<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FcdaNCJ%2FbtqAa7RVwhD%2FP60H1iai2Mxgvu09TGQpv0%2Fimg.png" width=80% height=80%/>

  - 즉, 배치 정규화는 배치 차원에 대해 정규화, 층 정규화는 <strong>특성 차원에 대해 정규화</strong>
  - 그래서 훈련과 테스트에서 동일한 방식으로 작동한다.
    - 배치 정규화는 훈련 세트의 모든 세트에 대한 특성 통계를 추정하기 위해 지수 이동 평균이 필요했음

## 메모리 셀 안에 층 정규화 구현

In [ ]:
class LNSimpleRNNCell(keras.layers.Layer):
  def __init__(self, units, activation="tanh", **kwargs):
    super().__init__(**kwargs)
    self.state_size=units
    self.output_size=units
    self.simple_rnn_cell=keras.layers.SimpleRNNCell(units,activation=None)#===포인트: activation=None이다(활성화 함수에 넣기 전에 정규화하려고)
    self.layer_norm=keras.layers.LayerNormalization()
    self.activation=keras.activations.get(activation)
  
  def call(self, inputs, states):#====현재 타임 스텝의 inputs와 이전 타임 스텝의 hidden states(h_t-1)
    outputs, new_states= self.simple_rnn_cell(inputs, states)
    norm_outputs=self.activation(self.layer_norm(outputs)) #정규화하고 난 후의 값을 활성화 함수에 대입
    return norm_outputs, [norm_outputs] #===두 개인 이유: 하나는 출력, 하나는 새로운 은닉 상태(h_t)가 된다


- 사용자 정의 셀 적용

In [ ]:
model=keras.models.Sequential([
                               keras.layers.RNN(LNSimpleRNNCell(20), return_sequences=True, input_shape=[None, 1]),
                               keras.layers.RNN(LNSimpleRNNCell(20), return_sequences=True),
                               keras.layers.TimeDistributed(keras.layers.Dense(10))
])

-----------
위 처럼 keras.layers.RNN층을 만들어서 LNSimpleRNNCell의 객체를 전달하면 된다.
- 만약 타임 스텝 사이에 dropout을 적용하고 싶다면?
  - 위처럼 드롭아웃을 적용하는 사용자 정의 셀을 만들어도 되긴하는데, 
  - keras.layers.SimpleRNN(recurrent_dropout=0.5) 매개변수를 지원함

## LSTM(Long Short-Term Memory) 셀
- RNN을 거치면서 데이터가 변환되므로 일부 정보는 매 훈련 스텝 후 사라짐
- 그래서 어느 정도 시간이 지나면 사실상 <strong>첫 번째 입력의 흔적이 사라짐</strong>
- 몇몇문제는 위 특징이 심각한 문제가 될 수 있음 그래서 <strong>장기 메모리</strong>를 가진 여러 종류의 셀이 연구됨
- 장기 메모리를 가진 셀에서 가장 인기있는게 <strong>LSTM셀</strong>

<img src="https://www.researchgate.net/profile/Xiaofeng-Yuan-4/publication/331421650/figure/fig2/AS:771405641695233@1560928845927/The-structure-of-the-LSTM-unit.png" width=50% height=50%/>

- 핵심 아이디어: 네트워크가 <strong>장기 상태에 저장</strong>할 것, <strong>버릴 것</strong>, 그리고 <strong>읽어들일 것</strong>을 학습하는 것
- h_t: 단기 상태(short-term state)
- c_t: 장기 상태(long-term state) 
- 로직
  - 장기기억 c_t-1은 왼->오른쪽으로 관통하면서 <strong>삭제 게이트(f_t)를 지나 일부 기억을 잃는다.</strong>
  - 그런 다음 <strong>입력 게이트(i_t)에서 새로운 기억 일부가 추가</strong>된다.
  - 만들어진 c_t는 다른 <strong>추가 변환 없이 바로 출력</strong>으로 보내진다.
  - h_t는 c_t가 복사되어 <strong>tanh함수</strong>를 거친 후 <strong>결과 게이트(o_t)</strong>에서 걸러서 만들어진다.
    - o_t는 장기 상태의 어느 부분을 읽어서 이 타임 스텝의 h_t와 y_t로 출력해야 하는지 제어한다.
<br><br><br>

### 적용 방법
- SImpleRNN 대신 LSTM 층 사용
  - 이 방법을 일반적으로 많이 사용(GPU에서 실행할 때 최적화된 구현을 사용해서)

In [ ]:
model2= keras.models.Sequential([
                                 keras.layers.LSTM(20,return_sequences=True, input_shape=[None, 1]),
                                 keras.layers.LSTM(20, return_sequences=True),
                                 keras.layers.TimeDistributed(keras.layers.Dense(10))
])

In [ ]:
model2.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, None, 20)          1760      
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 20)          3280      
_________________________________________________________________
time_distributed (TimeDistri (None, None, 10)          210       
Total params: 5,250
Trainable params: 5,250
Non-trainable params: 0
_________________________________________________________________


- RNN층에 LSTMCell을 매개변수로 지정할 수도 있음
  - 이 방법은 사용자 정의 셀을 정의할 때 많이 사용

In [ ]:
model3 = keras.models.Sequential([
                                  keras.layers.RNN(keras.layers.LSTMCell(20), return_sequences=True, input_shape=[None, 1]),
                                  keras.layers.RNN(keras.layers.LSTMCell(20), return_sequences=True),
                                  keras.layers.TimeDistributed(keras.layers.Dense(10))
])

In [ ]:
model3.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rnn_1 (RNN)                  (None, None, 20)          1760      
_________________________________________________________________
rnn_2 (RNN)                  (None, None, 20)          3280      
_________________________________________________________________
time_distributed_1 (TimeDist (None, None, 10)          210       
Total params: 5,250
Trainable params: 5,250
Non-trainable params: 0
_________________________________________________________________
